In [ ]:
pip install langchain langchain-experimental pymysql pymssql pyparsing openai tensorboard-data-server

In [ ]:
#pip install -U langchain-openai

In [ ]:
# We beginnen met de componenten te importeren
from langchain.sql_database import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate, ChatPromptTemplate
from langchain.schema import HumanMessage, SystemMessage


In [ ]:
# Importeer OpenAI om hun externe API te gebruiken ipv een lokale LLM
# ####################################################################
#from langchain.llms import OpenAI
#from langchain_openai import ChatOpenAI

# OPEN AI LLM initialiseren
#OPENAI_API_KEY = "<jouw OpenAI Key>"
#llm = ChatOpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

In [ ]:
# Importeer Ollama om de lokale LLM aan te spreken
# ###################################################################
from langchain_community.llms import Ollama

# Lokale LLM Gebruiken
llm = Ollama(
    temperature=0, 
    model="openchat" # codellama | mixtral 
)


In [ ]:
# LLM testen
# llm.invoke("Tell me a joke")

In [ ]:
# Bron: http://docs.sqlalchemy.org/en/latest/dialects/mssql.html 
# https://python.langchain.com/v0.1/docs/use_cases/sql/quickstart/
host = '127.0.0.1'
port = '1433'
username = 'afm'
password = 'afm'
database_schema = 'afmscm_202401'
mssql_uri = f"mssql+pymssql://{username}:{password}@{host}:{port}/{database_schema}"
db = SQLDatabase.from_uri(mssql_uri,include_tables=['afm_flds','afm_tbls'],sample_rows_in_table_info=1)
db_chain = SQLDatabaseChain.from_llm(llm, db, use_query_checker=True, verbose=True)

In [ ]:
db.run("SELECT top 1 afm_flds.table_name FROM afm_flds;")

In [ ]:
context=(
                """Context:
    All the database tables are stored in "afm_tbls" with: 
    table_name : primary key
    title : table name
    is_sql_view : is this an SQL view or table
    
    All the tables field names are stored in "afm_flds" table with the following columns per field:
    table_name : name of table to which the field belongs
    field_name : field name
    ml_heading : multi line heading also known and the title or name
    afm_size : the field length
    data_type : the data type of the field, which is an enumeration value 
    enum_list : possible enumeration values in key;value format
    ref_table : reference table validating the data via foreign keys
    pkey_value : value greater then 0 is a primary key of a table """
            )

In [ ]:
def ask_db(question: str) -> str:
    from langchain.chains import create_sql_query_chain
    db_chain = SQLDatabaseChain.from_llm(llm,db,use_query_checker=True, verbose=False)
    response = db_chain.run(context + question)
    return response

# Nu gaan we de database ondervragen

In [ ]:
ask_db("what is the multi line heading of the rm_id field in rm")

In [ ]:
ask_db("What is the enumeration list of the field prorate from table rm?")

In [ ]:
ask_db("list me the fields that are part of the primary key of the table fl")

In [ ]:
ask_db("what is the reference table of bl_id in the table rm?")